# Importing the data and saving it to .json

In [1]:
import pandas as pd
import numpy as np

In [2]:
source = "https://www.datos.gov.co/resource/vuyt-mqpw.json"
df = pd.read_json(source + "?$limit=600000")
df.to_json('../data/raw_data/raw_data.json')

# Adjustments for columns, nulls and duplicates, (cleaning process).

## Null data treatment

Renaming columns to English language

In [3]:
df.rename({'departamento':'department', 
           'municipio':'municipality', 
           'armas_medios':'gun_type', 
           'fecha_hecho':'date', 
           'genero':'gender', 
           'grupo_etario':'age_group', 
           'cantidad':'cases'}, axis = 1 ,inplace = True
)
df.drop('codigo_dane', axis = 1, inplace = True)


In [4]:
df.info(), df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564113 entries, 0 to 564112
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   department    564113 non-null  object
 1   municipality  564113 non-null  object
 2   gun_type      564113 non-null  object
 3   date          564113 non-null  object
 4   gender        564113 non-null  object
 5   age_group     562502 non-null  object
 6   cases         564113 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 30.1+ MB


(None,
 department      object
 municipality    object
 gun_type        object
 date            object
 gender          object
 age_group       object
 cases            int64
 dtype: object)

In [5]:
empty_info = df.isnull().sum()*100/df.shape[0]
print(f'{empty_info}\n\n{df.isnull().sum()}')

department      0.000000
municipality    0.000000
gun_type        0.000000
date            0.000000
gender          0.000000
age_group       0.285581
cases           0.000000
dtype: float64

department         0
municipality       0
gun_type           0
date               0
gender             0
age_group       1611
cases              0
dtype: int64


In [6]:
df = df[~df['age_group'].isnull()]   #Eliminates null values from 'age_group' column

Erasing duplicates

In [7]:
df = df.loc[:, ~df.columns.duplicated()].copy()

## Data dtype changes

In [8]:
df['gender'] = df['gender'].astype('category')
df['age_group'] = df['age_group'].astype('category')
df['gun_type'] = df['gun_type'].astype('category')
df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%Y')

In [9]:
df.dtypes

department              object
municipality            object
gun_type              category
date            datetime64[ns]
gender                category
age_group             category
cases                    int64
dtype: object

## Department column changes for feature graphs

In [10]:
df.loc[:, 'department'] = df['department'].str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8')

In [11]:
df['department'].replace({'SAN ANDRES':'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                             'VALLE':'VALLE DEL CAUCA',
                             'NARINO':'NARIÑO',
                             'GUAJIRA':'LA GUAJIRA'}, inplace = True)
df.department.unique()

array(['ATLANTICO', 'BOYACA', 'CAQUETA', 'CASANARE', 'CUNDINAMARCA',
       'SUCRE', 'VALLE DEL CAUCA', 'HUILA', 'ANTIOQUIA', 'ARAUCA',
       'BOLIVAR', 'CALDAS', 'CAUCA', 'CESAR', 'CHOCO', 'CORDOBA',
       'MAGDALENA', 'META', 'NARIÑO', 'NORTE DE SANTANDER', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'TOLIMA', 'VAUPES', 'GUAVIARE',
       'LA GUAJIRA', 'QUINDIO', 'AMAZONAS', 'VICHADA', 'GUAINIA',
       'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
       'NO REPORTA'], dtype=object)

In [12]:
df['department'].replace('NO REPORTA', np.nan, inplace = True)       #dropping data without department report
df = df[~df['department'].isnull()]

Changing the name of Bogotá municipality to plot maps later

In [13]:
df = df.replace('BOGOTÁ D.C. (CT)', 'SANTAFE DE BOGOTA D.C').copy()

## Little columns changes

In [14]:
print(df['gun_type'].unique())
print(df['gender'].unique())
print(df['age_group'].unique())

['ARMA BLANCA / CORTOPUNZANTE', 'ARMA DE FUEGO', 'CONTUNDENTES', 'CORTANTES', 'CORTOPUNZANTES', ..., 'PUNZANTES', 'SIN EMPLEO DE ARMAS', 'NO REPORTA', 'ESCOPOLAMINA', '-']
Length: 11
Categories (11, object): ['-', 'ARMA BLANCA / CORTOPUNZANTE', 'ARMA DE FUEGO', 'CONTUNDENTES', ..., 'NO REPORTA', 'NO REPORTADO', 'PUNZANTES', 'SIN EMPLEO DE ARMAS']
['MASCULINO', 'FEMENINO', 'NO REPORTA', 'NO REPORTADO']
Categories (4, object): ['FEMENINO', 'MASCULINO', 'NO REPORTA', 'NO REPORTADO']
['ADULTOS', 'ADOLESCENTES', 'MENORES', 'NO REPORTA', 'NO REPORTADO']
Categories (5, object): ['ADOLESCENTES', 'ADULTOS', 'MENORES', 'NO REPORTA', 'NO REPORTADO']


Artículo 365A del código penal colombiano

Parágrafo 1°. Para los efectos de esta ley, entiéndanse como arma blanca el objeto punzante, cortante, cortocontundente o cortopunzante apto para herir cortar, matar o dañar; que posea bordes filosos o punzantes, tales como navajas, puñales, puñaletas, punzones o cualquier objeto de similares características.

---

Article 365A of the Colombian Penal Code

Paragraph 1. For the purposes of this law, a white weapon is understood to be any sharp, cutting, blunt or sharp object suitable to wound, cut, kill or damage; that has sharp or pointed edges, such as knives, daggers, daggers, stabs, awls or any object of similar characteristics.

In [15]:
#change on genre no reports
df['gender'].replace('NO REPORTADO', 'NO REPORTA', inplace = True)

#change on age_group no reports
df['age_group'].replace('NO REPORTADO', 'NO REPORTA', inplace = True)

#change on gun_type based on Penal Code
df['gun_type'].replace({'ARMA BLANCA / CORTOPUNZANTE':'ARMA BLANCA',
                            '-':'NO REPORTA',
                            'NO REPORTADO':'NO REPORTA',
                            'CORTOPUNZANTES':'ARMA BLANCA',
                            'CORTANTES':'ARMA BLANCA',
                            'CONTUNDENTES':'ARMA BLANCA',
                            'PUNZANTES':'ARMA BLANCA'}, inplace = True)

# Exporting data as .parquet

In [17]:
df = df.reset_index(drop=True).copy()

In [16]:
df.to_parquet('../data/processed/data_cleaned.parquet', index = False)